In [26]:
import pandas as pd
import os
import numpy as np

# Ruta desde notebooks/ hacia los CSV
data_path = "../data/raw/"

# Listar archivos para confirmar que encuentra los CSV
archivos = os.listdir(data_path)
print("Archivos en la carpeta raw:", archivos)

#HOLIDAYS_EVENTS
holiday_df = pd.read_csv(os.path.join(data_path, "holidays_events.csv"))

#CONTROL DE FECHAS HOLIDAY

holiday_df["date"] = pd.to_datetime(holiday_df["date"])

fecha_min = holiday_df["date"].min()
fecha_max = holiday_df["date"].max()
 
nulos_date_holidays = holiday_df["date"].isnull().sum()

print(f"Los valores nulos en holidays.date son: {nulos_date_holidays}")
print(f" HOLIDAY: RANGO DE FECHAS DE {fecha_min} a {fecha_max}")


holiday_df["date"] = pd.to_datetime(holiday_df["date"])  # Asegurar que es datetime
holiday_df["year"] = holiday_df["date"].dt.year
holiday_df["month"] = holiday_df["date"].dt.month
holiday_df["day"] = holiday_df["date"].dt.day
holiday_df["day_of_week"] = holiday_df["date"].dt.weekday  # 0 = Lunes, 6 = Domingo
holiday_df["week_of_year"] = holiday_df["date"].dt.isocalendar().week  # Semana del año

#LIMPIEZA DE LA COLUMNA TRANSFERRED:

# Filtrar solo los registros donde transferred == False (es decir, NO transferidos)
holiday_df_clean = holiday_df[holiday_df.transferred == False]

# Eliminar la columna transferred, ya que ahora es innecesaria
holiday_df_clean = holiday_df_clean.drop("transferred", axis=1)

# Reiniciar el índice tras la limpieza
holiday_df_clean = holiday_df_clean.reset_index(drop=True)

#COMPROBACIÓN DE QUE ES CORRECTA

datos_iniciales = holiday_df.shape[0]

datosSinTransferred= holiday_df[holiday_df["transferred"] != False].shape[0]


holiday_df_clean = holiday_df[holiday_df["transferred"] == False]

datos_postFiltrado = holiday_df_clean.shape[0]

print(f"Los datos iniciales son: {datos_iniciales} los que hemos quitado {datosSinTransferred} los que nos quedan {datos_postFiltrado}  ")

nulos_date_holidays_clean = holiday_df_clean["date"].isnull().sum()


if nulos_date_holidays_clean > 0:
    raise ValueError(f"❌ ERROR: Existen {nulos_date_holidays_clean} valores nulos en la columna 'date' de holiday_df")

duplicados_holidays= holiday_df_clean.duplicated(subset=["date", "type", "locale", "locale_name","description"], keep=False)

if duplicados_holidays.any():
    raise ValueError(f"❌ ERROR: Existen {duplicados_holidays} valores duplicados en la clave única de holiday_df")
#OIL 

# Cargar y mostrar las primeras filas del archivo oil.csv
oil_df = pd.read_csv(os.path.join(data_path, "oil.csv"))


oil_df["date"] = pd.to_datetime(oil_df["date"])

#HAY NULOS EN DATE?

nulos_date_oil = oil_df["date"].isnull().sum()

print(f"Los valores nulos en oil.date son: {nulos_date_oil}")


if nulos_date_oil > 0:
    raise ValueError(f"❌ ERROR: Existen {nulos_date_oil} valores nulos en la columna 'date' de oil_df")

duplicados_oil= oil_df.duplicated(subset=["date"])

if duplicados_oil.any():
    raise ValueError(f"❌ ERROR: Existen {duplicados_oil} valores duplicados en la clave única de oil")

oil_df = oil_df.fillna(-1)  # Reemplaza NaN por -1


# STORES
stores = pd.read_csv(os.path.join(data_path, "stores.csv"))

store_nbr = stores["store_nbr"].isnull().sum()

print(f"Los valores nulos en stores.store_nbr son: {store_nbr}")

if store_nbr > 0:
    raise ValueError(f"❌ ERROR: Existen {store_nbr} valores nulos en la columna 'store_nbr' de stores")

duplicados_store = stores.duplicated(subset=["store_nbr"])

if duplicados_store.any():
    raise ValueError(f"❌ ERROR: Existen {duplicados_store} valores duplicados en la clave única de store")

#Transactions

transactions = pd.read_csv(os.path.join(data_path, "transactions.csv"))

transactions["date"] = pd.to_datetime(transactions["date"])

transactions_storenbr = transactions["store_nbr"].isnull().sum()
transactions_date = transactions["date"].isnull().sum()

print(f"Los valores nulos en transactions.store_nbr son: {transactions_storenbr}")
print(f"Los valores nulos en transactions.date son: {transactions_date}")

if ((transactions_storenbr > 0) and (transactions_date > 0)):
    raise ValueError(f"❌ ERROR: Existen {transactions_storenbr} valores nulos en la columna 'store_nbr' de transactions y Existen {transactions_date} valores nulos en la columna 'date' de transactions")

duplicados_transactions= transactions.duplicated(subset=["store_nbr","date"])

if duplicados_transactions.any():
    raise ValueError(f"❌ ERROR: Existen {duplicados_transactions} valores duplicados en la clave única de transactions")

base_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))  
processed_path = os.path.join(base_dir,"data","processed")

holiday_df_clean.to_csv(os.path.join(processed_path,"holiday_events.csv"),index=False)
oil_df.to_csv(os.path.join(processed_path,"oil.csv"),index=False)
stores.to_csv(os.path.join(processed_path,"stores.csv"),index=False)
transactions.to_csv(os.path.join(processed_path,"transactions.csv"),index=False)



Archivos en la carpeta raw: ['test.csv', 'train.csv', 'transactions.csv', 'oil.csv', 'holidays_events.csv', 'sample_submission.csv', 'stores.csv']
Los valores nulos en holidays.date son: 0
 HOLIDAY: RANGO DE FECHAS DE 2012-03-02 00:00:00 a 2017-12-26 00:00:00
Los datos iniciales son: 350 los que hemos quitado 12 los que nos quedan 338  
Los valores nulos en oil.date son: 0
Los valores nulos en stores.store_nbr son: 0
Los valores nulos en transactions.store_nbr son: 0
Los valores nulos en transactions.date son: 0


## Hacemos las querys recorriendo los distintos df y lo metemos en el script de arriba 

In [28]:
import pymysql
import pandas as pd
import os

# Conectar con MySQL
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="",  # Pon tu contraseña si la tienes
    database="store_sales"
)
cursor = conn.cursor()

# Definir ruta base y archivos
base_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
processed_path = os.path.join(base_dir, "data", "processed")

files = {
    "holiday_event": "holiday_events.csv",
    "transactions": "transactions.csv",
    "stores": "stores.csv",
    "oil": "oil.csv"
}

try:
    for table, filename in files.items():
        file_path = os.path.join(processed_path, filename)

        if not os.path.exists(file_path):
            raise FileNotFoundError(f"❌ ERROR: No se encontró el archivo {filename} en {file_path}")

        df = pd.read_csv(file_path)  # Cargar CSV dinámicamente

        # Construcción de la consulta según la tabla
        if table == "holiday_events":
            query = """
            INSERT IGNORE INTO holiday_events (date, type, locale, locale_name, description, transferred, year, month, day, day_week, week_year)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            data = [
                (
                    row["date"], row["type"], row["locale"], row["locale_name"], row["description"],
                    row["transferred"], row["year"], row["month"], row["day"],
                    row["day_of_week"], row["week_of_year"]
                ) for _, row in df.iterrows()
            ]
        elif table == "transactions":
            query = """
            INSERT IGNORE INTO transactions (date, store_nbr, transactions)
            VALUES (%s, %s, %s)
            """
            data = [(row["date"], row["store_nbr"], row["transactions"]) for _, row in df.iterrows()]

        elif table == "stores":
            query = """
            INSERT IGNORE INTO stores (store_nbr, city, state, type, cluster)
            VALUES (%s, %s, %s, %s, %s)
            """
            data = [(row["store_nbr"], row["city"], row["state"], row["type"], row["cluster"]) for _, row in df.iterrows()]

        elif table == "oil":
            query = """
            INSERT IGNORE INTO oil (date, price)
            VALUES (%s, %s)
            """
            data = [(row["date"], row["dcoilwtico"]) for _, row in df.iterrows()]

        # Iniciar transacción e insertar datos
        try:
            conn.begin()
            cursor.executemany(query, data)
            conn.commit()
            print(f"✅ Datos insertados correctamente en la tabla {table}.")
        except Exception as e:
            conn.rollback()
            raise ValueError(f"❌ ERROR al insertar en {table}: {e}")
except Exception as general_error:
   raise ValueError(f"❌ ERROR: Existen {general_error} valores nulos en la columna 'date' de oil_df")

finally:
    # Cerrar cursor y conexión para liberar recursos
    cursor.close()
    conn.close()
    print("🔌 Conexión a MySQL cerrada.")




✅ Datos insertados correctamente en la tabla holiday_event.
✅ Datos insertados correctamente en la tabla transactions.
✅ Datos insertados correctamente en la tabla stores.
✅ Datos insertados correctamente en la tabla oil.
🔌 Conexión a MySQL cerrada.
